## Download a colour map

This example shows how to download a colour map from an Evo workspace and inspect it's properties.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
from evo.colormaps import ColormapAPIClient
from evo.notebooks import ServiceManagerWidget
from evo.objects import ObjectAPIClient

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

client_id = "native-jupyter-app"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    base_uri="https://qa-ims.bentley.com",
    discovery_url="https://uat-api.test.seequent.systems",
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The colormap client will manage your colormap API requests.
colormap_client = ColormapAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Display all objects in the workspace

In [ ]:
from collections import defaultdict

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

all_objects = await object_client.list_all_objects()

if len(all_objects) == 0:
    print("No objects found.")
else:
    # Group objects by schema classification/sub-classification for a two-level picker
    def object_type(obj):
        sub_cls = getattr(obj.schema_id, "sub_classification", None)
        if isinstance(sub_cls, (list, tuple)) and sub_cls:
            return " / ".join(sub_cls)
        if isinstance(sub_cls, str) and sub_cls:
            return sub_cls
        cls = getattr(obj.schema_id, "classification", None)
        return cls or "Unknown"

    grouped = defaultdict(list)
    for obj in all_objects:
        grouped[object_type(obj)].append(obj)

    for objs in grouped.values():
        objs.sort(key=lambda o: o.name)

    type_options = sorted(grouped.keys())

    def build_object_options(selected_type):
        return [(o.name.removesuffix(".json"), str(o.id)) for o in grouped[selected_type]]

    # Dynamic widths based on content lengths
    def width_for_labels(labels, min_px=220, max_px=720, char_px=8):
        longest = max((len(lbl) for lbl in labels), default=0)
        return f"{min(max(min_px, longest * char_px + 40), max_px)}px"

    type_width = width_for_labels(type_options)
    object_width = width_for_labels([name for name, _ in build_object_options(type_options[0])])
    label_width = "110px"
    row_gap = "6px"

    type_dropdown = widgets.Dropdown(
        options=type_options,
        description="",
        disabled=False,
        layout=widgets.Layout(width=type_width),
    )

    objects_dropdown = widgets.Dropdown(
        options=build_object_options(type_options[0]),
        description="",
        disabled=False,
        layout=widgets.Layout(width=object_width),
    )

    def on_type_change(change):
        if change.get("name") == "value" and change.get("new") in grouped:
            new_options = build_object_options(change["new"])
            objects_dropdown.options = new_options
            objects_dropdown.layout.width = width_for_labels([name for name, _ in new_options])

    type_dropdown.observe(on_type_change, names="value")

    label_layout = Layout(width=label_width, display="flex", justify_content="flex-start", align_items="center")
    row_layout = Layout(align_items="center", gap=row_gap)
    type_row = widgets.HBox([widgets.Label("Object type", layout=label_layout), type_dropdown], layout=row_layout)
    objects_row = widgets.HBox([widgets.Label("Objects", layout=label_layout), objects_dropdown], layout=row_layout)

    print(f"Found {len(all_objects)} object(s) across {len(type_options)} type(s). Select a type, then an object:")
    display(widgets.VBox([type_row, objects_row]))

### Fetch colormap data

Find all colormap associations for the chosen object.

In [ ]:
import json
from datetime import datetime
from uuid import UUID

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer


def serialize_for_display(obj):
    """Convert an object's attributes to a JSON-serializable dictionary."""
    if isinstance(obj, UUID):
        return str(obj)
    elif isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_for_display(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_for_display(value) for key, value in obj.items()}
    elif hasattr(obj, "__dict__"):
        return {key: serialize_for_display(value) for key, value in vars(obj).items()}
    else:
        return obj


object_id = objects_dropdown.value
associations = await colormap_client.get_association_collection(object_id)
print("\nColormap associations for this object:\n")
for association in associations:
    serialized = serialize_for_display(association)
    print(highlight(json.dumps(serialized, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

Iterate through all colormaps to learn what the colormap type is, eg. category, continuous, or discrete, then cross-reference the colormap `attribute ID` with the attributes in the chosen geoscience object.

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Prepare a combined list of collections from planned and interim data.
downloaded_object = await object_client.download_object_by_id(object_id=object_id)

downloaded_dict = downloaded_object.as_dict()
collections = downloaded_dict.get("planned", {}).get("collections", []) + downloaded_dict.get("interim", {}).get(
    "collections", []
)

for association in associations:
    colormap_id = association.colormap_id

    colormap_metadata = await colormap_client.get_colormap_by_id(colormap_id)

    print(f"\n{'=' * 80}")
    print(f"Colormap ID: {colormap_id}")
    print(f"{'=' * 80}\n")

    # Cross-reference the attribute ID with the drilling campaign planned and interim data to find the attribute name.
    attribute_id = association.attribute_id
    attribute_name = None

    # Search through collections to find the attribute with matching ID
    for collection in collections:
        if "from_to" in collection and "attributes" in collection["from_to"]:
            for attr in collection["from_to"]["attributes"]:
                if attr.get("key") == attribute_id:
                    attribute_name = attr.get("name")
                    break
        if attribute_name:
            break

    if attribute_name:
        print(f"Associated with attribute: {attribute_name}")
    else:
        print(f"Attribute ID: {attribute_id} (name not found)")

    colormap = colormap_metadata.colormap
    colormap_type = colormap.__class__.__name__
    print(f"Colormap type: {colormap_type}\n")

    if colormap_type == "CategoryColormap":
        map = colormap.map
        colors = colormap.colors

        # Normalize colors from 0-255 range to 0-1 range for matplotlib
        normalized_colors = [tuple(c / 255.0 for c in color) for color in colors]

        # Create visualization
        fig, ax = plt.subplots(figsize=(10, len(colors) * 0.5))
        ax.set_xlim(0, 1)
        ax.set_ylim(0, len(colors))
        ax.axis("off")

        for i, (label, color) in enumerate(zip(map, normalized_colors)):
            y_pos = len(colors) - i - 1
            # Draw color rectangle
            rect = mpatches.Rectangle((0, y_pos), 0.15, 0.8, facecolor=color, edgecolor="black", linewidth=0.5)
            ax.add_patch(rect)
            # Add label text
            ax.text(0.18, y_pos + 0.4, f"{label}", va="center", fontsize=10)
            print(f"{label}: {colors[i]}")

        plt.title(f"Category Colormap: {attribute_name or 'Unknown'}", fontsize=12, pad=10, loc="left")
        plt.tight_layout()
        plt.show()

    elif colormap_type == "DiscreteColormap":
        gradient_controls = colormap.gradient_controls
        colors = colormap.colors

        print("Gradient controls and colors:")
        for control, color in zip(gradient_controls, colors):
            print(f"{control}: {color}")

        # Create stepped gradient visualization
        fig, ax = plt.subplots(figsize=(12, 2))

        # Normalize colors from 0-255 range to 0-1 range for matplotlib
        normalized_colors = np.array([[tuple(c / 255.0 for c in color) for color in colors]])

        # Display as image with each color taking equal space
        ax.imshow(normalized_colors, aspect="auto", extent=[gradient_controls[0], gradient_controls[-1], 0, 1])

        # Add vertical lines and labels at control points
        for control, color in zip(gradient_controls, colors):
            ax.axvline(x=control, color="black", linewidth=0.5, alpha=0.5)
            ax.text(control, -0.3, f"{control:.2f}", ha="center", va="top", fontsize=9)

        ax.set_yticks([])
        ax.set_xlabel("Value", fontsize=10)
        plt.title(f"Discrete Colormap: {attribute_name or 'Unknown'}", fontsize=12, pad=10)
        plt.tight_layout()
        plt.show()

    elif colormap_type == "ContinuousColormap":
        attribute_controls = colormap.attribute_controls
        gradient_controls = colormap.gradient_controls
        colors = colormap.colors

        print(f"Attribute controls: {attribute_controls}")
        print(f"Gradient controls: {gradient_controls}")
        print(f"Number of colors: {len(colors)}")

        # Create continuous gradient visualization
        fig, ax = plt.subplots(figsize=(12, 2))

        # Normalize colors from 0-255 range to 0-1 range for matplotlib
        normalized_colors = [tuple(c / 255.0 for c in color) for color in colors]

        # gradient_controls define the data value range [min, max]
        # Colors should be spread evenly across this range
        min_val = gradient_controls[0]
        max_val = gradient_controls[-1]

        # Create evenly spaced positions for the colors across the gradient range
        num_colors = len(colors)
        if num_colors == 1:
            # Single color - place it at the midpoint
            color_positions = [min_val]
        else:
            # Spread colors evenly from min to max
            color_positions = [min_val + (max_val - min_val) * i / (num_colors - 1) for i in range(num_colors)]

        # Normalize positions to 0-1 range for matplotlib colormap creation
        normalized_positions = [(pos - min_val) / (max_val - min_val) for pos in color_positions]
        color_list = list(zip(normalized_positions, normalized_colors))

        # Create custom colormap
        cmap = LinearSegmentedColormap.from_list("custom", color_list)

        # Create gradient array
        gradient = np.linspace(0, 1, 256).reshape(1, -1)
        ax.imshow(gradient, aspect="auto", cmap=cmap, extent=[min_val, max_val, 0, 1])

        # Add markers at color anchor positions without text labels to avoid overlap
        # for pos, color in zip(color_positions, normalized_colors):
        #     ax.axvline(x=pos, color="black", linewidth=0.5, alpha=0.5, linestyle="--")
        #     ax.plot(pos, 0.5, "o", color="black", markersize=6, markerfacecolor=color, markeredgewidth=1)

        ax.set_yticks([])
        plt.title(f"Continuous Colormap: {attribute_name or 'Unknown'}", fontsize=12, pad=10)
        plt.tight_layout()
        plt.show()

    else:
        print("Unknown colormap type")